In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # For visualization
from sklearn.ensemble import RandomForestClassifier  # Random Forest for bagging
from sklearn.tree import DecisionTreeClassifier  # Decision Tree model
from sklearn.tree import export_graphviz  # To visualize the decision tree
from sklearn.metrics import roc_curve, auc  # ROC and AUC curve for model evaluation
from sklearn.metrics import classification_report  # Classification report for decision trees
from sklearn.metrics import confusion_matrix  # Confusion matrix
from sklearn.model_selection import train_test_split  # Split dataset into train and test sets
from scipy.stats import pearsonr
import shap  # For SHAP values
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from scipy.interpolate import splev, splrep
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

In [7]:
from autogluon.tabular import TabularDataset,TabularPredictor

In [8]:
df = pd.read_csv(r'D:\002NationalPark\001Research1\003Analysis\training_data.csv', index_col=0)
df.head(5)

,RM_dist,MW_dist,GS_dist,XV_dist,CR_dist,RR_dist,Water_dist,NDVI,TEM,DEM,Slope,Population,NPP,Vegetation,Builtup_dist,present
152944,342918.9051,404663.1268,29624.23884,40538.79691,40545.09897,86625.40564,36399.33869,0.456,16.699446,3424,1.308435,3,0.120827,4991,46913.04826,0
155641,342723.5518,404198.8228,28425.56963,39372.05467,39373.67436,85365.28314,35540.80878,0.820,18.192223,3422,0.448084,5,0.124442,4991,46875.21836,0
155582,342242.8113,403762.5789,28653.59779,39558.17487,39565.64348,85676.09911,35570.02076,0.696,16.202084,3412,0.971957,4,0.132564,4991,46181.95585,0
155642,342049.1866,403299.7177,27448.97353,38387.67814,38390.38065,84413.65187,34710.90552,0.860,19.672916,3420,0.695848,4,0.138532,4991,46165.19301,0
155692,341859.5800,402841.2894,26265.25517,37231.74061,37231.74061,83156.08989,33870.65747,0.712,20.113195,3409,2.029433,5,0.157077,4991,46191.74217,0


In [10]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('present',1),df['present'],test_size=0.3,random_state = 10)

In [11]:
X = df
y = df['present']
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=0.3,random_state=10)

In [12]:
print(test_x.shape)
print(train_x.shape)
print(test_y.shape)

(50948, 16)
(118878, 16)
(50948,)


In [13]:
predictor = TabularPredictor(label='present').fit(train_data=train_x)

No path specified. Models will be saved in: "AutogluonModels\ag-20240313_020422\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240313_020422\"
AutoGluon Version:  0.7.0
Python Version:     3.8.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Train Data Rows:    118878
Train Data Columns: 15
Label Column: present
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 

[1000]	valid_set's binary_error: 0.0316
[2000]	valid_set's binary_error: 0.022
[3000]	valid_set's binary_error: 0.0196


	0.982	 = Validation score   (accuracy)
	13.03s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0184
[2000]	valid_set's binary_error: 0.0152


	0.9868	 = Validation score   (accuracy)
	7.74s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9784	 = Validation score   (accuracy)
	25.17s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9788	 = Validation score   (accuracy)
	26.98s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.9852	 = Validation score   (accuracy)
	108.7s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9804	 = Validation score   (accuracy)
	4.71s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9792	 = Validation score   (accuracy)
	4.78s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9752	 = Validation score   (accuracy)
	110.88s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	0.9832	 = Validation score   (accuracy)
	6.8s	 = Training   runtime
	0.0

In [14]:
leaderboard = predictor.leaderboard()

                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.9912       0.205789  342.243610                0.003991           1.618629            2       True         14
1        KNeighborsUnif     0.9872       0.041887    0.662229                0.041887           0.662229            1       True          1
2        KNeighborsDist     0.9868       0.025390    0.897589                0.025390           0.897589            1       True          2
3              LightGBM     0.9868       0.048870    7.739233                0.048870           7.739233            1       True          4
4              CatBoost     0.9852       0.009930  108.697506                0.009930         108.697506            1       True          7
5         LightGBMLarge     0.9844       0.028858    5.076613                0.028858           5.076613            1       True         13
6               XGBo

In [30]:
type(leaderboard)
leaderboard.to_csv(r'D:\002NationalPark\001Research1\003Analysis\20231120Revision_model_compare\autogluon_leaderboard.csv')

In [26]:
predictor.feature_importance(data=train_x)

Computing feature importance via permutation shuffling for 15 features using 5000 rows with 5 shuffle sets...
	38.76s	= Expected runtime (7.75s per shuffle set)
	33.31s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
RM_dist,0.24452,0.003817,7.125591e-09,5,0.252380,0.236660
MW_dist,0.22076,0.005033,3.238923e-08,5,0.231122,0.210398
XV_dist,0.18468,0.007884,3.975187e-07,5,0.200913,0.168447
Water_dist,0.09732,0.004863,7.458713e-07,5,0.107334,0.087306
RR_dist,0.09404,0.003378,1.993440e-07,5,0.100994,0.087086
Builtup_dist,0.07816,0.002647,1.576766e-07,5,0.083611,0.072709
CR_dist,0.07348,0.001906,5.425090e-08,5,0.077404,0.069556
GS_dist,0.05740,0.003581,1.807424e-06,5,0.064772,0.050028
DEM,0.00416,0.001322,1.075358e-03,5,0.006882,0.001438
Vegetation,0.00184,0.000767,2.912319e-03,5,0.003419,0.000261
